In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd

import numpy as np
import scipy.stats as stats

In [13]:
def sample(d0, d1, n=32):
    x0 = d0.sample((n,))
    x1 = d1.sample((n,))
    y0 = torch.zeros((n, 1))
    y1 = torch.ones((n, 1))
    return torch.cat([x0, x1], 0), torch.cat([y0, y1], 0)

In [14]:
d0 = torch.distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
d1 = torch.distributions.MultivariateNormal(torch.ones(2), torch.eye(2))

In [15]:
layer = nn.Linear(2, 1)
print([p.data[0] for p in layer.parameters()])

[tensor([-0.1528,  0.3009]), tensor(1.00000e-02 *
       -8.9133)]


In [ ]:
log_freq = 500
for i in range(100000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1, 100000)
            output = F.binary_cross_entropy_with_logits(layer(x), y)
            loss = output.item()
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    x, y = sample(d0, d1, 100000)
    layer.zero_grad()
    output = F.binary_cross_entropy_with_logits(layer(x), y)
    loss = output.item()

    output.backward()
    for param in layer.parameters():
        param.data.add_(-0.1 * param.grad.data)

Ошибка после 0 итераций: 0.670761
